In [ ]:
import tensorflow as tf
import random

# Setting seeds (Optional)
random.seed(42)
tf.random.set_seed(42)

In [ ]:
# HYPERPARAMETERS
NUM_SAMPLES = 1000
lr = 0.1
mse_loss = tf.keras.losses.MeanSquaredError()

# Helper function to create dummy data
def create_dummy_data(num_samples):
  x = tf.expand_dims(tf.constant([complex(i/num_samples,i/num_samples) for i 
                                  in range(num_samples)], tf.complex128), -1)
  # f(x): x -> 5x
  y = tf.expand_dims(tf.constant([complex(5*(j/num_samples) + random.random(),
                                          5*(j/num_samples) + random.random()) for j 
                                  in range(num_samples)], tf.complex128), -1)
  return x,y

In [ ]:
# Helper function for complex optimization
def train_complex(train_x, train_y, test_x, test_y, w):
  for i in range(31):
    
    # Shuffling at the start of every epoch
    indices = tf.range(start=0, limit=train_x.shape[0], dtype=tf.int32)
    shuffled_indices = tf.random.shuffle(indices)
    train_x = tf.gather(train_x, shuffled_indices)
    train_y = tf.gather(train_y, shuffled_indices)

    with tf.GradientTape() as tape:
      # Get y_pred. Linear activation is used for this example
      y_pred = tf.matmul(train_x, tf.cast(w, tf.complex128)) 
    
      # Get real valued loss
      mse = mse_loss(train_y, y_pred)
    
    if i % 10 == 0:
      val_loss = mse_loss(test_y, tf.matmul(test_x, tf.cast(w, tf.complex128)))
      print(f"Training Loss at epoch {i}: {tf.abs(mse).numpy()}, Validation Loss: {tf.abs(val_loss).numpy()}")

    # Get gradients
    dL_dw = tape.gradient(mse, w)

    # Apply raw backprop
    w.assign(w - dL_dw * lr)

  print("Training finished")

In [ ]:
# Helper function for real optimization (split kernel approach)
def train_real(train_x, train_y, val_x, val_y, w_real, w_imag):

  # Splitting validation data into real and imaginary parts
  val_x_real, val_x_imag = tf.math.real(val_x), tf.math.imag(val_x)
  val_y_real, val_y_imag = tf.math.real(val_y), tf.math.imag(val_y)

  for i in range(61):

    # Shuffling at the start of every epoch
    indices = tf.range(start=0, limit=train_x.shape[0], dtype=tf.int32)
    shuffled_indices = tf.random.shuffle(indices)
    train_x = tf.gather(train_x, shuffled_indices)
    train_y = tf.gather(train_y, shuffled_indices)

    # Splitting real and imaginary parts from shuffled data
    x_real, x_imag = tf.math.real(train_x), tf.math.imag(train_x)
    y_real, y_imag = tf.math.real(train_y), tf.math.imag(train_y)

    with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
      # Get y_pred for real and imaginary parts separately
      y_pred_real = tf.matmul(x_real, tf.cast(w_real, tf.float64)) 
      y_pred_imag = tf.matmul(x_imag, tf.cast(w_imag, tf.float64)) 


      # Calculate real valued losses
      mse_real = mse_loss(y_real, y_pred_real)
      mse_imag = mse_loss(y_imag, y_pred_imag)
    
    if i % 10 == 0:
      val_pred_y_real = tf.matmul(val_x_real, tf.cast(w_real, tf.float64)) 
      val_pred_y_imag = tf.matmul(val_x_imag, tf.cast(w_imag, tf.float64)) 
      val_loss = mse_loss(val_y_real, val_pred_y_real) + mse_loss(val_y_imag, val_pred_y_imag)
      print(f"Training Loss at epoch {i}: {mse_real.numpy() + mse_imag.numpy()}, Validation Loss: {val_loss.numpy()}")

    
    # Get separate gradients
    dL_dw_r = tape1.gradient(mse_real, w_real)
    dL_dw_i = tape2.gradient(mse_imag, w_imag)


    # Apply raw backprop on both components
    w_real.assign(w_real - dL_dw_r * lr)
    w_imag.assign(w_imag - dL_dw_i * lr)


  print("Training finished")

In [ ]:
# Creating dummy data
x, y  = create_dummy_data(NUM_SAMPLES)
# Train test split (80%-20%)
train_x, test_x, train_y, test_y = x[:int(.8*NUM_SAMPLES), :], x[int(.8*NUM_SAMPLES):, :], y[:int(.8*NUM_SAMPLES),:], y[int(.8*NUM_SAMPLES):,:]

# Initializing weights
w = tf.Variable(tf.zeros((1,1), tf.complex128), tf.complex128)
# Training complex optimization example
train_complex(train_x, train_y, test_x, test_y, w)

# Intializing two seprate kernels for real and imaginary domains
w_real, w_imag = tf.Variable(tf.zeros((1,1), tf.float64)), tf.Variable(tf.zeros((1,1), tf.float64))
# Training real optimization example
train_real(train_x, train_y, test_x, test_y, w_real, w_imag)